# Test Colecciones

In [1]:
import requests
import json
from pymongo import MongoClient

import sys
sys.path.append("../")

from funciones_queries import *

ModuleNotFoundError: No module named 'funciones_queries'

In [ ]:
client = MongoClient()
db = client.get_database("kamino")
aloj = client.kamino.Alojamiento
rest = client.kamino.Restauracion
patr = client.kamino.Patrimonio

In [ ]:
list(db.list_collections())

## Test Alojamiento

In [ ]:
#Todos los alojaminetos por localidad
q1 = {"LOCALIDAD":"PAMPLONA"}

z1 = list(db.Alojamiento.find(q1,{"_id":0, "CÓDIGO":0}))

z1[:5]

In [ ]:
# Todos los alojamiento de una localidad y mostrando que tipo de alojamiento
q2 = {"LOCALIDAD":"PAMPLONA"}

z2 = list(db.Alojamiento.find(q2,{"_id":0, "MODALIDAD":1, "NOMBRE":1}))

z2[:10]

In [ ]:
# Todos los alojamiento de una localidad por modalidad y mostrando su nombre
q3 = {"$and":[{"LOCALIDAD":"PAMPLONA"},{"MODALIDAD":"Hotel"}]}

z3 = list(db.Alojamiento.find(q3,{"_id":0, "MODALIDAD":1, "NOMBRE":1, "LOCALIDAD":1}))

z3[:5]

In [ ]:
# Todos los alojamiento de una localidad por modalidad, mostrando su nombre Y dirección
q4 = {"$and":[{"LOCALIDAD":"PAMPLONA"},{"MODALIDAD":"Hotel"}]}

z4 = list(db.Alojamiento.find(q4,{"_id":0, "MODALIDAD":1, "NOMBRE":1, "LOCALIDAD":1, "DIRECCIÓN":1}))

z4[:5]

## Test Restauración

In [ ]:
#Todas las opciones de Restauración por localidad
q1 = {"LOCALIDAD":"ESTELLA"}

z1 = list(db.Restauracion.find(q1,{"_id":0, "CÓDIGO":0}))

z1[:5]

In [ ]:
# Toda la restauración de una localidad y mostrando qué tipo y su nombre
q2 = {"LOCALIDAD":"ESTELLA"}

z2 = list(db.Restauracion.find(q2,{"_id":0, "MODALIDAD":1, "NOMBRE":1}))

z2[:10]

In [ ]:
# Todos los restaurantes de una localidad por modalidad y mostrando su nombre y localidad
q3 = {"$and":[{"LOCALIDAD":"ESTELLA"},{"MODALIDAD":"Restaurantes"}]}

z3 = list(db.Restauracion.find(q3,{"_id":0, "MODALIDAD":1, "NOMBRE":1, "LOCALIDAD":1}))

z3[:5]

In [ ]:
# Todos los Restaurantes de una localidad por modalidad, mostrando su nombre, dirección y localidad
q4 = {"$and":[{"LOCALIDAD":"ESTELLA"},{"MODALIDAD":"Restaurantes"}]}

z4 = list(db.Restauracion.find(q4,{"_id":0, "MODALIDAD":1, "NOMBRE":1, "LOCALIDAD":1, "DIRECCIÓN":1}))

z4[:5]

## Test Patrimonio

In [ ]:
#Todas las opciones de Patrimonio por localidad
q1 = {"LOCALIDAD":"PUENTE LA REINA"}

z1 = list(db.Patrimonio.find(q1,{"_id":0, "CÓDIGO":0}))

z1[:5]

In [ ]:
# Todo el patrimonio de una localidad y 
q2 = {"LOCALIDAD":"PUENTE LA REINA"}

z2 = list(db.Patrimonio.find(q2,{"_id":0, "CATEGORÍA":1, "NOMBRE":1}))

z2[:10]

In [ ]:
# Todos el patrimonio de una localidad y mostrando nombre, categoría y estilo
q3 = {"LOCALIDAD":"PUENTE LA REINA"}

z3 = list(db.Patrimonio.find(q3,{"_id":0,"CATEGORÍA":1, "NOMBRE":1, "ESTILO":1}))

z3[:5]

In [ ]:
# Todo el patrimonio de una localidad mostrando sus coordenadas
q4 = {"LOCALIDAD":"PUENTE LA REINA"}

z4 = list(db.Patrimonio.find(q4,{"_id":0, "NOMBRE":1, "LOCALIDAD":1, "COORDENADAS":1}))

z4[:5]

# Funciones y mapas

In [ ]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from geopy.geocoders import Nominatim

In [ ]:
def localidad():
    
    """
    La función recibe un input del usuario; la localización en la que se encuentra y a través de Geocoder
    devuelve el nombre de la localidad y las coordenadas.
    
    Input: Nombre de la localidad
    Output: Nombre de la localidad y las coordenadas de la localidad
    
    """
    
    localidad = (input("Localidad: ")).upper() #Convierte el input en mayúsculas para que es´te igual que en la BBDD.
    
    locator = Nominatim(user_agent = "myGeocoder")
    location = locator.geocode(localidad)
    coordenadas = location.latitude, location.longitude
 
    return localidad, list(coordenadas)

In [ ]:
localidad()

In [ ]:
localidad()

In [ ]:
def locali_colec_coord(coleccion, *lugar):
    
    """
    La función recibe una colección y un lugar; hace una query a la colección seleccionada y el lugar
    especifivado y devuelve las coordenadas de los lugares encontrados.
   
    
    Input: Nombre de la colección y localidad
    Output: Las coordenadas de la localidad
    
    """
    bbdd = db[f'{coleccion}']
    query = {"LOCALIDAD":f"{lugar[0]}"}
    coordenadas = list(bbdd.find(query,{"_id":0,"COORDENADAS":1}))
    return coordenadas

In [ ]:
locali_colec_coord("Patrimonio", "PAMPLONA")[:2]

In [ ]:
locali_colec_coord("Alojamiento", "PAMPLONA")[:5]

In [ ]:
locali_colec_coord("Alojamiento", "ESTELLA")[:5]

In [ ]:
locali_colec_coord("Alojamiento", "ESTELLA")[:10]

In [ ]:
locali_colec_coord("Restauracion", "PAMPLONA")[:5]

In [ ]:
def mapa (coleccion):
    
    """
    Función creada para mostrar en un mapa la información relacionada con cada colección dependiendo de la localidad
    
    input : localidad y colección
    output : mapa
    
    
    """
    
    nombre, punto_inicio = localidad() #Devuelve el nombre de la localidad y las coordenadas
    
    query = locali_colec_coord(coleccion, nombre) #Hace una query con el nombre de la localidad
    
    mapa = folium.Map(location = punto_inicio, zoom_start=15) #Devuelve un mapa con las coordenadas
    

    for cada_entrada in query:
        coordenadas = list(cada_entrada.values())
        lat = coordenadas[0].split(",")[0][1:]
        lon = coordenadas[0].split(",")[1][:-1]
                         
        loc = {"location":[lat,lon],
          "tooltip": f"{coleccion}"}

        marker_of = Marker(**loc, icon = markers(coleccion) , popup=f"{coleccion}")
        marker_of.add_to(mapa)
    
    return mapa

In [ ]:
mapa("Alojamiento")

In [ ]:
def markers(coleccion):
    
    
    """ 
    Función que devuelve un icono según la colección.
    
    Input : Nombre de la colecciñon
    Output : Icono
    
    
    
    """
    
    if coleccion == "Alojamiento":
        icono = Icon(color = "blue",
                prefix = "fa",
                icon = "hotel",
                icon_color = "black")
        
    elif coleccion == "Patrimonio":   
        icono = Icon(color = "orange",
                prefix = "fa",
                icon = "camera",
                icon_color = "black")
        
    else:
        icono = Icon(color = "green",
                prefix = "fa",
                icon = "spoon",
                icon_color = "black")

    return icono

In [ ]:
!git remote show origin

In [ ]:
mapa("Alojamiento")

In [ ]:
mapa("Patrimonio")

In [ ]:
mapa("Patrimonio")

In [ ]:
mapa("Restauracion")

In [ ]:
mapa("Patrimonio")